# Factor Analysis on SVF Port, Spring 2024

**Steps:**

1. get yfinance hist data for all stocks in the portfolio
2. get fama french factor returns
3. compute monthly returns df for port holdings
4. align factor and returns data
5. run a linear factor model

In [92]:
# imports
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import yfinance as yf
import pandas_datareader.data as web
from linearmodels.asset_pricing import LinearFactorModel

In [180]:
STOX = ["CLBT", "BLBD", "NFE", "GXO", "BELFB", "ULCC", "OCSL"]

# get yfinance data for each of the stocks for the last two years
yf_data = {x: None for x in STOX}

for stock in STOX:
    # download last 24 mo of data
    yf_data[stock] = yf.download(stock, start="2021-01-01", interval="1mo")

# test
yf_data["OCSL"].head()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-01-01,16.83,17.549999,16.410000,16.680000,12.180859,3465668
2021-02-01,17.10,18.809999,16.559999,18.450001,13.473432,4825402
2021-03-01,18.60,19.080000,18.209999,18.600000,13.582972,5798801
2021-04-01,18.57,20.490000,18.570000,19.980000,14.874057,8342532
2021-05-01,20.07,20.490000,19.110001,20.219999,15.052722,4818400


In [181]:
ocsl_data = yf.Ticker("OCSL").history(start="2021-01-01", interval="1mo")
dividends = ocsl_data["Dividends"]
ocsl_df = yf_data["OCSL"]
ocsl_df["Adj Close"] = ocsl_df["Close"] # some error going on with yf
for date, div in dividends.items():
    if div > 0:
        # drop timezone
        date = date.tz_localize(None)
        ocsl_df.loc[date:, "Adj Close"] *= (1 + div / ocsl_df.loc[date, "Close"])

yf_data["OCSL"] = ocsl_df

In [182]:
# get fama french factor returns
ff_factor = 'F-F_Research_Data_5_Factors_2x3'
ff_factor_data = web.DataReader(ff_factor, 'famafrench', start="2021-01-01", end="2023-12-31")[0]
ff_factor_data = ff_factor_data.iloc[8:]
ff_factor_data.head()

,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,
2021-09,-4.37,1.12,5.08,-1.96,2.10,0.00
2021-10,6.65,-2.70,-0.49,1.66,-1.45,0.00
2021-11,-1.55,-1.77,-0.45,7.20,1.73,0.00
2021-12,3.10,-0.80,3.26,4.91,4.40,0.01
2022-01,-6.25,-4.06,12.75,0.84,7.72,0.00


In [183]:
# compute monthly returns dataframe for the portfolio
monthly_returns = pd.DataFrame(index=yf_data["CLBT"].index, columns=STOX)
for stock in STOX:
    monthly_returns[stock] = yf_data[stock]["Adj Close"].pct_change()

monthly_returns = monthly_returns.dropna()
monthly_returns = monthly_returns.iloc[:len(monthly_returns) - 2]
monthly_returns.head()

,CLBT,BLBD,NFE,GXO,BELFB,ULCC,OCSL
Date,,,,,,,
2021-09-01,-0.096585,-0.036490,-0.055158,-0.040959,-0.119688,0.030007,-0.017053
2021-10-01,0.206263,-0.059923,0.084647,0.132075,0.119871,-0.008866,0.043909
2021-11-01,-0.301701,0.032126,-0.170333,0.081644,-0.130066,-0.146965,-0.001357
2021-12-01,0.028205,-0.227273,-0.030133,-0.054347,0.073920,0.016479,0.034647
2022-01-01,-0.251870,-0.008312,-0.086603,-0.105912,-0.039443,-0.036109,0.009383


In [184]:
excess_returns = pd.DataFrame(index=monthly_returns.index, columns=STOX)
for stock in STOX:
    excess_returns[stock] = monthly_returns[stock] - ff_factor_data["RF"].values

ff_factor_data.drop(columns=["RF"], inplace=True)
excess_returns.head()

,CLBT,BLBD,NFE,GXO,BELFB,ULCC,OCSL
Date,,,,,,,
2021-09-01,-0.096585,-0.036490,-0.055158,-0.040959,-0.119688,0.030007,-0.017053
2021-10-01,0.206263,-0.059923,0.084647,0.132075,0.119871,-0.008866,0.043909
2021-11-01,-0.301701,0.032126,-0.170333,0.081644,-0.130066,-0.146965,-0.001357
2021-12-01,0.018205,-0.237273,-0.040133,-0.064347,0.063920,0.006479,0.024647
2022-01-01,-0.251870,-0.008312,-0.086603,-0.105912,-0.039443,-0.036109,0.009383


In [194]:
# run the linear factor model

mod = LinearFactorModel(portfolios=excess_returns, factors=ff_factor_data)
results = mod.fit()
print(results.full_summary)

                      LinearFactorModel Estimation Summary                      
No. Test Portfolios:                  7   R-squared:                      0.2546
No. Factors:                          5   J-statistic:                    10.763
No. Observations:                    27   P-value                         0.0046
Date:                  Fri, Jan 12 2024   Distribution:                  chi2(2)
Time:                          15:13:58                                         
Cov. Estimator:                  robust                                         
                                                                                
                            Risk Premia Estimates                             
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Mkt-RF         1.6723     3.3297     0.5022     0.6155     -4.8538      8.1984
SMB           -3.3661     2.5263    

In [202]:
premias = results.risk_premia

# plot the loadings
fig = go.Figure()
fig.add_trace(go.Bar(x=premias.index, y=premias.values))
fig.update_layout(title="(Unweighted) Portfolio FF Risk Premia", xaxis_title="Factor", yaxis_title="Premia")
fig.update_traces(marker_color='rgb(175, 0, 0)')
fig.show()

# Weighted portfolio analysis

In [196]:
# Compute the portfolio's monthly returns

ASSET_WEIGHTS = [0.163, 0.1604, 0.0897, 0.0721, 0.0315, 0.1162, 0.1953]
portfolio_returns = monthly_returns.dot(ASSET_WEIGHTS)
portfolio_returns

# Extract the loadings for each individual stock
loadings_df = results.params
loadings_df.head(10)

,alpha,Mkt-RF,SMB,HML,RMW,CMA
CLBT,-0.022653,0.009786,0.036066,-0.005203,-0.001767,0.015399
BLBD,-0.001949,0.004779,0.032664,-0.010555,0.001367,0.017684
NFE,-0.003802,0.014389,0.011692,-0.003141,-0.020801,0.037370
GXO,-0.024989,0.011775,0.021851,-0.003901,0.013156,0.017618
BELFB,0.034128,0.011470,0.025933,-0.009837,0.001098,0.020907
ULCC,0.010079,0.017884,0.047089,0.009098,0.011387,0.012993
OCSL,0.006010,0.001367,0.018549,-0.000643,0.009994,0.020155


In [188]:
portfolio_loadings = pd.DataFrame(columns=loadings_df.columns)
for col in portfolio_loadings.columns:
    loading = 0
    for i in range(len(STOX)):
        loading += loadings_df.loc[STOX[i], col] * ASSET_WEIGHTS[i]
    portfolio_loadings[col] = [loading]

portfolio_loadings.head()

,alpha,Mkt-RF,SMB,HML,RMW,CMA
0,-0.002728,0.007208,0.023654,-0.002482,0.002324,0.016074


In [189]:
# Plot the portfolio loadings
fig = go.Figure()
fig.add_trace(go.Bar(x=portfolio_loadings.columns, y=portfolio_loadings.values[0]))
fig.update_layout(title="Portfolio Fama-French Factor Loadings", xaxis_title="Factor", yaxis_title="Loading")
# make the bars dark red
fig.update_traces(marker_color='rgb(175, 0, 0)')
fig.show()

# Industry Factor Analysis

In [204]:
# get fama french factor returns
industry_factor = '5_Industry_Portfolios'
industry_factor_data = web.DataReader(industry_factor, 'famafrench', start="2021-01-01", end="2023-12-31")[0]
industry_factor_data = industry_factor_data.iloc[8:]
industry_factor_data.head()

,Cnsmr,Manuf,HiTec,Hlth,Other
Date,,,,,
2021-09,-3.28,-2.84,-6.16,-6.01,-2.26
2021-10,8.63,6.01,6.74,2.25,7.25
2021-11,0.51,-1.85,0.35,-4.29,-4.96
2021-12,1.54,5.86,1.94,6.62,4.88
2022-01,-7.48,-0.86,-7.86,-8.77,-3.22


In [205]:
# run the linear factor model

mod = LinearFactorModel(portfolios=monthly_returns, factors=industry_factor_data)
results = mod.fit()
print(results.full_summary)

                      LinearFactorModel Estimation Summary                      
No. Test Portfolios:                  7   R-squared:                      0.4896
No. Factors:                          5   J-statistic:                    4.8736
No. Observations:                    27   P-value                         0.0874
Date:                  Fri, Jan 12 2024   Distribution:                  chi2(2)
Time:                          15:20:55                                         
Cov. Estimator:                  robust                                         
                                                                                
                            Risk Premia Estimates                             
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Cnsmr          3.5012     2.7151     1.2895     0.1972     -1.8204      8.8228
Manuf          0.7797     3.2231    

In [208]:
premias = results.risk_premia

# plot the loadings
fig = go.Figure()
fig.add_trace(go.Bar(x=premias.index, y=premias.values))
fig.update_layout(title="(Unweighted) Portfolio FF Risk Premia", xaxis_title="Factor", yaxis_title="Premia")
fig.update_traces(marker_color='rgb(175, 0, 0)')
fig.show()

In [206]:
portfolio_industry_loadings = pd.DataFrame(columns=results.params.columns)
for col in portfolio_industry_loadings.columns:
    loading = 0
    for i in range(len(STOX)):
        loading += results.params.loc[STOX[i], col] * ASSET_WEIGHTS[i]
    portfolio_industry_loadings[col] = [loading]

portfolio_industry_loadings.head()

,alpha,Cnsmr,Manuf,HiTec,Hlth,Other
0,0.00464,0.002877,0.000306,0.002044,-0.004208,0.008881


In [207]:
# Plot the portfolio loadings
fig = go.Figure()
fig.add_trace(go.Bar(x=portfolio_industry_loadings.columns, y=portfolio_industry_loadings.values[0]))
fig.update_layout(title="Portfolio Industry Factor Loadings", xaxis_title="Factor", yaxis_title="Loading")
fig.update_traces(marker_color='rgb(175, 0, 0)')
fig.show()

# Benchmark Analysis

In [218]:
# bring in the portfolio
portfolio = pd.read_excel('../portfolio_building/data/portfolio_value.xlsx', index_col=0)
portfolio = portfolio.loc["2020-01-01":]
portfolio.head()

,value
2020-01-01,278356.872263
2020-01-02,278607.595764
2020-01-03,278214.285008
2020-01-04,278214.285008
2020-01-05,278467.954897


In [219]:
# portfolio pctchange
portfolio_pctchange = portfolio.pct_change()
portfolio_pctchange = portfolio_pctchange.dropna()
portfolio_pctchange.head()

,value
2020-01-02,0.000901
2020-01-03,-0.001412
2020-01-04,0.000000
2020-01-05,0.000912
2020-01-06,0.000000


In [220]:
start_date = min(portfolio_pctchange.index)
end_date = max(portfolio_pctchange.index)

spy_df = yf.download("SPY", start=start_date, end=end_date)
russell_df = yf.download("IWM", start=start_date, end=end_date)
nasdaq_df = yf.download("QQQ", start=start_date, end=end_date)

# delete dates in portfolio that are not in the spy_df
for date in portfolio_pctchange.index:
    if date not in spy_df.index:
        portfolio_pctchange.drop(date, inplace=True)

print(len(portfolio_pctchange.index))
print(len(spy_df.index))
print(len(russell_df.index))
print(len(nasdaq_df.index))

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
1014
1014
1014
1014


In [221]:
spy_pctchange = spy_df["Adj Close"].pct_change()
russell_pctchange = russell_df["Adj Close"].pct_change()
nasdaq_pctchange = nasdaq_df["Adj Close"].pct_change()

In [236]:
# Plot the cumulative returns

cumulative_returns = pd.DataFrame(index=portfolio_pctchange.index, columns=["Portfolio", "SPY", "Russell", "Nasdaq"])
cumulative_returns["Portfolio"] = portfolio_pctchange.cumsum()
cumulative_returns["SPY"] = spy_pctchange.cumsum()
cumulative_returns["Russell"] = russell_pctchange.cumsum()
cumulative_returns["Nasdaq"] = nasdaq_pctchange.cumsum()

fig = go.Figure()
fig.add_trace(go.Scatter(x=cumulative_returns.index, y=cumulative_returns["Portfolio"], name="SVF"))
fig.add_trace(go.Scatter(x=cumulative_returns.index, y=cumulative_returns["SPY"], name="S&P 500"))
fig.add_trace(go.Scatter(x=cumulative_returns.index, y=cumulative_returns["Russell"], name="Russell"))
fig.add_trace(go.Scatter(x=cumulative_returns.index, y=cumulative_returns["Nasdaq"], name="Nasdaq"))
fig.update_layout(title="Cumulative Returns", xaxis_title="Date", yaxis_title="Cumulative Return")
fig.update_traces(line=dict(color='rgb(200, 0, 0)', width=3), selector=dict(name="SVF"))
fig.update_layout(font=dict(family="Courier New, monospace", size=18, color="#7f7f7f"))
# make chart a square
fig.update_layout(
    autosize=False,
    width=1250,
    height=800,
)
fig.show()